In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: Synthetic Driving Scenario Generation with Video Diffusion
## Implementation Notebook -- Meridian Autonomy

*You are an ML engineer at Meridian Autonomy, an autonomous trucking startup. Your perception models fail on rare driving scenarios (dust storms, tire debris, nighttime construction). Real data for these edge cases costs USD 18,000 per scenario-hour. Your task: build a video diffusion model that generates realistic dashcam clips conditioned on driving conditions, so the perception team can train on synthetic data instead.*

*This notebook uses real dashcam video from the KITTI dataset -- the industry-standard benchmark for autonomous driving perception. You will download actual driving sequences, build a factorized video diffusion model, train it on real road footage, and generate synthetic driving clips.*

*Estimated time: 75-90 minutes on a T4 GPU*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/diffusion-models-video-generation/practice/0/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
import os
import zipfile
import io
import time
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
torch.manual_seed(42)
np.random.seed(42)

## 1. Data Acquisition: Real Dashcam Video from KITTI

The KITTI dataset contains real dashcam video captured from a car driving through Karlsruhe, Germany. Each sequence has color camera frames at 10 fps -- the same framerate Meridian's trucks use. We download several sequences spanning city streets, residential areas, and highways.

**Why KITTI?** It is the most widely used autonomous driving benchmark in the research community. Using it connects your work to real AV perception pipelines. The sequences contain real vehicles, pedestrians, cyclists, lane markings, traffic signs, and diverse road conditions -- exactly what Meridian's synthetic data must replicate.

### Download KITTI Raw Sequences

We download 6 driving sequences directly from the KITTI servers. Each zip contains a folder of PNG frames from the front-facing color camera. Total download is approximately 500 MB.

In [ ]:
KITTI_BASE = "https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data"

# 6 sequences with diverse driving conditions
SEQUENCES = {
    # City driving with traffic, pedestrians, parked vehicles
    "2011_09_26_drive_0005": {"date": "2011_09_26", "desc": "city - moderate traffic"},
    "2011_09_26_drive_0014": {"date": "2011_09_26", "desc": "city - heavy traffic, turns"},
    "2011_09_26_drive_0019": {"date": "2011_09_26", "desc": "city - traffic lights, intersections"},
    # Residential streets with parked cars, trees, quieter roads
    "2011_09_26_drive_0001": {"date": "2011_09_26", "desc": "residential - parked cars, trees"},
    "2011_09_26_drive_0009": {"date": "2011_09_26", "desc": "residential - narrow streets"},
    # Highway/road driving with fewer obstacles, higher speed
    "2011_09_26_drive_0015": {"date": "2011_09_26", "desc": "road - open road, few vehicles"},
}

DATA_DIR = Path("/content/kitti_raw")
DATA_DIR.mkdir(exist_ok=True)

for seq_name, info in SEQUENCES.items():
    seq_dir = DATA_DIR / info["date"] / f"{seq_name}_sync" / "image_02" / "data"
    if seq_dir.exists() and len(list(seq_dir.glob("*.png"))) > 10:
        print(f"  {seq_name} already downloaded ({info['desc']})")
        continue

    url = f"{KITTI_BASE}/{seq_name}/{seq_name}_sync.zip"
    print(f"  Downloading {seq_name} ({info['desc']})...")

    try:
        import urllib.request
        urllib.request.urlretrieve(url, f"/content/{seq_name}.zip")
        with zipfile.ZipFile(f"/content/{seq_name}.zip", 'r') as z:
            z.extractall(str(DATA_DIR))
        os.remove(f"/content/{seq_name}.zip")
        n_frames = len(list(seq_dir.glob("*.png")))
        print(f"    -> {n_frames} frames extracted")
    except Exception as e:
        print(f"    -> Download failed: {e}")
        print(f"       Visit https://www.cvlibs.net/datasets/kitti/raw_data.php")
        print(f"       Download '{seq_name}_sync.zip' and upload to /content/kitti_raw/")

print("\nDone. If any downloads failed, see instructions above.")

### Preprocess: Extract Video Clips

We extract 8-frame clips from each sequence. Each clip is a short dashcam video at 10 fps (0.8 seconds of driving). We resize to 64x64 for training efficiency -- small enough to train on a T4 GPU, but large enough to preserve road structure, vehicles, and lane markings.

We also assign each clip a **scene label** based on which sequence it came from:
- Label 0-2: City driving (traffic, intersections, pedestrians)
- Label 3-4: Residential (parked cars, trees, narrow streets)
- Label 5: Open road (highway-like, fewer obstacles)

These labels serve as the conditioning signal for our diffusion model -- analogous to Meridian's text prompts like "dust storm" or "nighttime construction."

In [ ]:
CLIP_LENGTH = 8       # frames per clip
CLIP_STRIDE = 4       # stride between clips (overlapping extraction)
FRAME_SIZE = 64       # resize to 64x64
N_CLASSES = 6         # number of scene condition labels

# Map sequence index to class label
SEQ_NAMES = list(SEQUENCES.keys())

def extract_clips_from_sequence(seq_name, seq_info, label):
    """Extract 8-frame clips from a single KITTI sequence."""
    frame_dir = DATA_DIR / seq_info["date"] / f"{seq_name}_sync" / "image_02" / "data"
    if not frame_dir.exists():
        print(f"  Skipping {seq_name}: not found")
        return [], []

    # Load all frames from this sequence
    frame_paths = sorted(frame_dir.glob("*.png"))
    frames = []
    for fp in frame_paths:
        img = Image.open(fp).convert('RGB')
        img = img.resize((FRAME_SIZE, FRAME_SIZE), Image.LANCZOS)
        arr = np.array(img, dtype=np.float32) / 255.0  # (H, W, 3)
        frames.append(arr)
    frames = np.stack(frames)  # (N_frames, H, W, 3)

    # Extract overlapping clips
    clips = []
    labels = []
    for start in range(0, len(frames) - CLIP_LENGTH, CLIP_STRIDE):
        clip = frames[start:start + CLIP_LENGTH]  # (8, 64, 64, 3)
        clips.append(clip)
        labels.append(label)

    return clips, labels


all_clips = []
all_labels = []

for idx, (seq_name, seq_info) in enumerate(SEQUENCES.items()):
    clips, labels = extract_clips_from_sequence(seq_name, seq_info, label=idx)
    all_clips.extend(clips)
    all_labels.extend(labels)
    print(f"  {seq_name} ({seq_info['desc']}): {len(clips)} clips, label={idx}")

all_clips = np.stack(all_clips)    # (N, 8, 64, 64, 3)
all_labels = np.array(all_labels)  # (N,)

# Convert to torch: (N, T, C, H, W)
videos_tensor = torch.tensor(all_clips).permute(0, 1, 4, 2, 3).float()
labels_tensor = torch.tensor(all_labels).long()

print(f"\nTotal clips: {len(videos_tensor)}")
print(f"Tensor shape: {videos_tensor.shape}")
print(f"Labels: {dict(zip(*np.unique(all_labels, return_counts=True)))}")

### Visualize Real Dashcam Clips

Let us look at actual clips from each driving condition. Each row shows 8 consecutive frames (0.8 seconds of driving). Notice how the scene changes: vehicles move, road geometry shifts as the car turns, and the visual context differs between city, residential, and open road.

In [ ]:
fig, axes = plt.subplots(N_CLASSES, 8, figsize=(20, 15))
fig.suptitle("Real KITTI Dashcam Clips (each row = 1 clip, columns = consecutive frames)",
             fontsize=14, fontweight='bold')

conditions = ["City: moderate traffic", "City: heavy traffic", "City: intersections",
              "Residential: parked cars", "Residential: narrow", "Open road"]

for cls in range(N_CLASSES):
    mask = all_labels == cls
    if not mask.any():
        continue
    idx = np.where(mask)[0][len(np.where(mask)[0]) // 2]  # pick middle clip
    clip = videos_tensor[idx]  # (T, C, H, W)
    for t in range(8):
        frame = clip[t].permute(1, 2, 0).numpy()  # (H, W, 3)
        axes[cls, t].imshow(frame)
        axes[cls, t].axis('off')
        if t == 0:
            axes[cls, t].set_ylabel(conditions[cls], fontsize=9, rotation=0,
                                     labelpad=100, va='center')

plt.tight_layout()
plt.show()

## 2. Exploratory Data Analysis

Before building the model, we analyze the temporal dynamics and condition distributions in our real driving data. This is directly relevant to Meridian: understanding how motion and appearance vary across conditions tells us what the generative model must learn.

### TODO 1: Analyze Temporal Dynamics Across Driving Conditions

Compute temporal statistics per driving condition. This tells us how much motion varies between city driving (lots of stop-and-go, turns) vs. open road (steady motion). The diffusion model must capture these differences.

In [ ]:
def analyze_temporal_dynamics(videos: torch.Tensor, labels: torch.Tensor) -> dict:
    """
    Compute per-condition temporal statistics for driving video clips.

    Args:
        videos: (N, T, C, H, W) real dashcam video clips in [0, 1]
        labels: (N,) integer class labels for each clip

    Returns:
        Dictionary with keys:
            'per_condition_motion': dict mapping label (int) to mean optical flow
                magnitude (float). Optical flow proxy: mean absolute pixel
                difference between consecutive frames.
            'per_condition_brightness': dict mapping label (int) to mean pixel
                brightness (float).
            'class_counts': dict mapping label (int) to count (int).

    Steps:
        1. For each unique label in labels:
           a. Select all clips with that label
           b. Compute frame-to-frame differences: |v[:, t+1] - v[:, t]|
              Average across all pixels, frames, and clips for that label.
              This gives mean_motion for that condition.
           c. Compute mean brightness: average of all pixel values for that label
        2. Compute class counts
        3. Return the results

    Hints:
        - (videos[mask][:, 1:] - videos[mask][:, :-1]).abs().mean() for motion
        - videos[mask].mean() for brightness
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return {
        'per_condition_motion': {},
        'per_condition_brightness': {},
        'class_counts': {},
    }

In [ ]:
# Verification
stats = analyze_temporal_dynamics(videos_tensor, labels_tensor)
assert len(stats['per_condition_motion']) > 0, "Should have motion stats"
assert len(stats['class_counts']) > 0, "Should have class counts"

conditions = ["City: moderate", "City: heavy", "City: intersections",
              "Residential: parked", "Residential: narrow", "Open road"]

print("Driving Condition Analysis:")
print(f"{'Condition':<25} {'Clips':>6} {'Motion':>8} {'Brightness':>10}")
print("-" * 55)
for cls in sorted(stats['class_counts'].keys()):
    print(f"{conditions[cls]:<25} {stats['class_counts'][cls]:>6} "
          f"{stats['per_condition_motion'].get(cls, 0):>8.4f} "
          f"{stats['per_condition_brightness'].get(cls, 0):>10.4f}")
print("\nTODO 1 passed.")

In [ ]:
# Visualize the condition distribution (this is the long-tail problem!)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Class distribution
counts = [stats['class_counts'].get(i, 0) for i in range(N_CLASSES)]
colors = ['#e74c3c', '#e67e22', '#f1c40f', '#2ecc71', '#3498db', '#9b59b6']
ax1.barh(range(N_CLASSES), counts, color=colors)
ax1.set_yticks(range(N_CLASSES))
ax1.set_yticklabels(conditions, fontsize=9)
ax1.set_xlabel("Number of Clips")
ax1.set_title("Distribution of Driving Conditions")
ax1.invert_yaxis()

# Motion per condition
motions = [stats['per_condition_motion'].get(i, 0) for i in range(N_CLASSES)]
ax2.barh(range(N_CLASSES), motions, color=colors)
ax2.set_yticks(range(N_CLASSES))
ax2.set_yticklabels(conditions, fontsize=9)
ax2.set_xlabel("Mean Frame-to-Frame Difference")
ax2.set_title("Temporal Motion by Condition")
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

**Think about this:** In Meridian's real system, the rare scenarios (dust storms, nighttime construction) have far fewer training clips than normal highway driving. How does this class imbalance affect training a conditional diffusion model? What techniques could mitigate it?

## 3. Baseline: Why Frame-Independent Generation Fails

Before building the full video model, let us demonstrate the core problem: generating each frame independently produces temporally incoherent video. This is exactly why Meridian cannot simply use an image diffusion model to generate driving scenarios.

### Build a Simple Image Diffusion Model

In [ ]:
class SimpleImageUNet(nn.Module):
    """Minimal 2D U-Net for single-frame diffusion (baseline)."""

    def __init__(self, in_channels=3, dim=64, time_dim=128):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_dim), nn.SiLU(), nn.Linear(time_dim, dim)
        )
        self.enc1 = nn.Sequential(nn.Conv2d(in_channels, dim, 3, padding=1), nn.SiLU())
        self.enc2 = nn.Sequential(nn.Conv2d(dim, dim*2, 3, stride=2, padding=1), nn.SiLU())
        self.bot = nn.Sequential(nn.Conv2d(dim*2, dim*2, 3, padding=1), nn.SiLU())
        self.dec2 = nn.Sequential(nn.ConvTranspose2d(dim*4, dim, 4, stride=2, padding=1), nn.SiLU())
        self.dec1 = nn.Conv2d(dim*2, in_channels, 3, padding=1)

    def forward(self, x, t):
        t_emb = self.time_mlp(t.float().unsqueeze(-1))
        h1 = self.enc1(x) + t_emb[:, :, None, None]
        h2 = self.enc2(h1)
        h = self.bot(h2)
        h = self.dec2(torch.cat([h, h2], dim=1))
        return self.dec1(torch.cat([h, h1], dim=1))


def cosine_noise_schedule(timesteps=200):
    """Cosine noise schedule (Nichol & Dhariwal, 2021)."""
    s = 0.008
    steps = torch.arange(timesteps + 1, dtype=torch.float64)
    f = torch.cos((steps / timesteps + s) / (1 + s) * (torch.pi / 2)) ** 2
    alphas_cumprod = (f / f[0])[:timesteps]
    betas = 1 - alphas_cumprod[1:] / alphas_cumprod[:-1]
    betas = torch.clamp(betas, 0.0001, 0.9999)
    alphas = 1 - betas
    alphas_cumprod = torch.cumprod(alphas, dim=0)
    return {
        'betas': betas.float(),
        'alphas': alphas.float(),
        'alphas_cumprod': alphas_cumprod.float(),
    }


noise_schedule = cosine_noise_schedule(200)

### Train on Individual Dashcam Frames

In [ ]:
image_model = SimpleImageUNet(in_channels=3).to(device)
optimizer = optim.Adam(image_model.parameters(), lr=2e-4)
ac = noise_schedule['alphas_cumprod'].to(device)

# Flatten all video clips into individual frames
all_frames = videos_tensor.reshape(-1, 3, FRAME_SIZE, FRAME_SIZE)  # (N*T, 3, H, W)
frame_loader = DataLoader(all_frames, batch_size=64, shuffle=True, drop_last=True)

print(f"Training image baseline on {len(all_frames)} individual dashcam frames...")
for epoch in range(5):
    total_loss = 0
    for batch in frame_loader:
        batch = batch.to(device)
        t = torch.randint(0, 199, (batch.shape[0],), device=device)
        noise = torch.randn_like(batch)
        ab = ac[t].reshape(-1, 1, 1, 1)
        noisy = torch.sqrt(ab) * batch + torch.sqrt(1 - ab) * noise
        loss = F.mse_loss(image_model(noisy, t), noise)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"  Epoch {epoch+1}: Loss = {total_loss / len(frame_loader):.4f}")

print("Image baseline trained.")

### TODO 2: Generate Videos Frame-by-Frame and Measure Coherence

Generate 8-frame driving clips by sampling each frame independently from the image model. Then measure how temporally incoherent these videos are compared to real KITTI clips.

In [ ]:
@torch.no_grad()
def generate_independent_video(model, noise_schedule, n_frames=8, size=64):
    """
    Generate a video by sampling each frame independently from an image model.

    Args:
        model: Pretrained 2D U-Net for single frames
        noise_schedule: Dict with 'betas', 'alphas', 'alphas_cumprod'
        n_frames: Number of frames to generate
        size: Frame height/width

    Returns:
        Tensor of shape (n_frames, 3, size, size) in [0, 1]

    Steps:
        1. For each of n_frames:
           a. Start with pure noise x_T ~ N(0, I) of shape (1, 3, H, W)
           b. Denoise via the full DDPM reverse loop (t = T-1 down to 0):
              eps = model(x_t, t_tensor)
              mean = (1/sqrt(alpha_t)) * (x_t - beta_t/sqrt(1 - alpha_bar_t) * eps)
              if t > 0: x_{t-1} = mean + sqrt(beta_t) * noise
              else: x_{t-1} = mean
        2. Stack all frames, clamp to [0, 1]

    Hints:
        - Move schedule tensors to the correct device
        - t_tensor should be shape (1,) with dtype long
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return torch.zeros(n_frames, 3, size, size)


def temporal_coherence_score(video: torch.Tensor) -> float:
    """
    Measure temporal coherence as mean cosine similarity between consecutive frames.

    Args:
        video: (T, C, H, W) tensor

    Returns:
        Float in [-1, 1], higher = more temporally coherent

    Steps:
        1. Flatten each frame: (T, C*H*W)
        2. For consecutive pairs, compute cosine similarity
        3. Return the mean
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return 0.0

In [ ]:
# Verification: compare real vs independently generated
print("Generating 5 independent dashcam clips...")
indep_scores = []
for i in range(5):
    vid = generate_independent_video(image_model, noise_schedule)
    score = temporal_coherence_score(vid.cpu())
    indep_scores.append(score)
    print(f"  Generated clip {i}: coherence = {score:.3f}")

real_scores = [temporal_coherence_score(videos_tensor[i]) for i in range(5)]

print(f"\nReal KITTI clips:     mean coherence = {np.mean(real_scores):.3f}")
print(f"Independent gen:      mean coherence = {np.mean(indep_scores):.3f}")
print(f"Gap: {np.mean(real_scores) - np.mean(indep_scores):.3f}")
print("\nThis gap is why Meridian cannot use frame-by-frame generation.")
print("TODO 2 passed." if np.mean(indep_scores) < np.mean(real_scores) else "TODO 2 needs work.")

**Key insight for Meridian:** A perception model that trains on temporally incoherent synthetic video would learn to track objects that randomly teleport between frames -- the exact opposite of what we need. The object tracker would produce fragmented tracks, destroying the training signal. This is why we need joint video generation.

## 4. Building the Video Diffusion Model

Now we build the core model: a factorized video U-Net that jointly generates all 8 frames with temporal coherence. This is the same architecture used in VDM, Stable Video Diffusion, and Imagen Video -- scaled to fit our KITTI data on a T4 GPU.

### Multi-Head Self-Attention (Shared Module)

This exact same module handles both spatial and temporal attention. The difference is only in how we reshape the input.

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    """Standard multi-head self-attention, reused for spatial and temporal."""

    def __init__(self, dim, n_heads=4):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = dim // n_heads
        self.qkv = nn.Linear(dim, 3 * dim)
        self.out = nn.Linear(dim, dim)
        self.scale = self.head_dim ** -0.5

    def forward(self, x):
        B, S, D = x.shape
        qkv = self.qkv(x).reshape(B, S, 3, self.n_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(B, S, D)
        return self.out(out)

### TODO 3: Spatial Attention Block

Spatial attention processes each frame independently. The key reshape: merge the time axis into the batch dimension so each frame becomes a separate item in the batch. Frame 1 cannot see frame 2 -- this handles "what each frame looks like."

In [ ]:
class SpatialAttentionBlock(nn.Module):
    """Self-attention within each frame independently."""

    def __init__(self, dim, n_heads=4):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.attn = MultiHeadSelfAttention(dim, n_heads)

    def forward(self, x):
        """
        Args:
            x: (B, T, H, W, C) video features

        Returns:
            (B, T, H, W, C) with spatial attention applied per frame

        Steps:
            1. Save input as residual
            2. Apply layer norm
            3. Get shape: B, T, H, W, C = x.shape
            4. Reshape to (B*T, H*W, C) -- each frame is independent
            5. Apply self-attention
            6. Reshape back to (B, T, H, W, C)
            7. Add residual and return

        Hints:
            - x.reshape(B*T, H*W, C) for step 4
            - x.reshape(B, T, H, W, C) for step 6
        """
        # ============ TODO ============
        #
        # YOUR CODE HERE
        #
        # ==============================
        return x

### TODO 4: Temporal Attention Block

Temporal attention processes across frames at each spatial position. The key reshape: merge spatial dimensions into the batch so each pixel position attends across time. This handles "how things move between frames."

In [ ]:
class TemporalAttentionBlock(nn.Module):
    """Self-attention across frames at each spatial position."""

    def __init__(self, dim, n_heads=4):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.attn = MultiHeadSelfAttention(dim, n_heads)

    def forward(self, x):
        """
        Args:
            x: (B, T, H, W, C) video features

        Returns:
            (B, T, H, W, C) with temporal attention applied per position

        Steps:
            1. Save input as residual
            2. Apply layer norm
            3. Get shape: B, T, H, W, C = x.shape
            4. Permute to (B, H, W, T, C), reshape to (B*H*W, T, C)
            5. Apply self-attention
            6. Reshape to (B, H, W, T, C), permute back to (B, T, H, W, C)
            7. Add residual and return

        Hints:
            - x.permute(0, 2, 3, 1, 4) for step 4
            - After attn: reshape(B, H, W, T, C).permute(0, 3, 1, 2, 4) for step 6
        """
        # ============ TODO ============
        #
        # YOUR CODE HERE
        #
        # ==============================
        return x

In [ ]:
# Verification
test_x = torch.randn(2, 4, 8, 8, 64)
sp = SpatialAttentionBlock(64)
tp = TemporalAttentionBlock(64)

assert sp(test_x).shape == test_x.shape, "Spatial block shape mismatch"
assert tp(test_x).shape == test_x.shape, "Temporal block shape mismatch"

# Verify spatial independence: changing frame 2 must not affect frame 0
x = torch.randn(1, 4, 8, 8, 64)
out1 = sp(x.clone())
x_mod = x.clone()
x_mod[:, 2] = torch.randn(1, 8, 8, 64)
out2 = sp(x_mod)
assert torch.allclose(out1[:, 0], out2[:, 0], atol=1e-5), \
    "Spatial attention is leaking across frames!"

print("TODO 3 and TODO 4 passed -- attention blocks verified.")

### Factorized Space-Time Block + Conditional Video U-Net

Now we combine spatial attention, temporal attention, and a feedforward network into a single factorized block, then build the full U-Net.

In [ ]:
class FactorizedSpaceTimeBlock(nn.Module):
    """Spatial attention -> Temporal attention -> FFN"""

    def __init__(self, dim, n_heads=4, ff_mult=4):
        super().__init__()
        self.spatial = SpatialAttentionBlock(dim, n_heads)
        self.temporal = TemporalAttentionBlock(dim, n_heads)
        self.ffn = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim * ff_mult),
            nn.GELU(),
            nn.Linear(dim * ff_mult, dim),
        )

    def forward(self, x):
        x = self.spatial(x)
        x = self.temporal(x)
        return x + self.ffn(x)


class ConditionalVideoUNet(nn.Module):
    """
    Video U-Net with factorized attention and driving-condition conditioning.
    Uses 2D convolutions (per-frame) + factorized attention for temporal coherence.
    Conditioned on driving condition labels (city/residential/open road).
    """

    def __init__(self, in_ch=3, base_dim=64, n_classes=6, time_dim=128, n_heads=4):
        super().__init__()
        # Time embedding (diffusion timestep)
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_dim), nn.SiLU(), nn.Linear(time_dim, base_dim)
        )
        # Condition embedding (driving condition label; -1 = unconditional)
        self.cond_emb = nn.Embedding(n_classes + 1, base_dim)

        # Encoder (2D conv per frame)
        self.enc1 = nn.Sequential(nn.Conv2d(in_ch, base_dim, 3, padding=1), nn.SiLU())
        self.enc2 = nn.Sequential(
            nn.Conv2d(base_dim, base_dim * 2, 3, stride=2, padding=1), nn.SiLU()
        )

        # Bottleneck: factorized attention (operates at half resolution)
        self.attn = FactorizedSpaceTimeBlock(base_dim * 2, n_heads)

        # Decoder
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(base_dim * 4, base_dim, 4, stride=2, padding=1), nn.SiLU()
        )
        self.dec1 = nn.Conv2d(base_dim * 2, in_ch, 3, padding=1)

    def forward(self, x, t, labels):
        """
        Args:
            x: (B, C, T, H, W) noisy video
            t: (B,) diffusion timesteps
            labels: (B,) condition labels (-1 for unconditional)
        """
        B, C, T, H, W = x.shape
        t_emb = self.time_mlp(t.float().unsqueeze(-1))
        c_emb = self.cond_emb(labels + 1)  # shift: -1->0, 0->1, etc.
        cond = t_emb + c_emb

        # Process each frame with 2D convs
        x_flat = x.permute(0, 2, 1, 3, 4).reshape(B * T, C, H, W)
        cond_bt = cond.unsqueeze(1).expand(-1, T, -1).reshape(B * T, -1)

        h1 = self.enc1(x_flat) + cond_bt[:, :, None, None]
        h2 = self.enc2(h1)

        # Factorized attention at bottleneck
        _, C2, H2, W2 = h2.shape
        h = h2.reshape(B, T, C2, H2, W2).permute(0, 1, 3, 4, 2)
        h = self.attn(h)
        h = h.permute(0, 1, 4, 2, 3).reshape(B * T, C2, H2, W2)

        # Decode
        h = self.dec2(torch.cat([h, h2], dim=1))
        h = self.dec1(torch.cat([h, h1], dim=1))
        return h.reshape(B, T, C, H, W).permute(0, 2, 1, 3, 4)

In [ ]:
# Test the model
model = ConditionalVideoUNet(n_classes=N_CLASSES).to(device)
test_v = torch.randn(2, 3, 8, FRAME_SIZE, FRAME_SIZE).to(device)
test_t = torch.randint(0, 200, (2,)).to(device)
test_l = torch.tensor([0, 3]).to(device)
out = model(test_v, test_t, test_l)

n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,}")
print(f"Input shape:  {test_v.shape}")
print(f"Output shape: {out.shape}")
assert out.shape == test_v.shape
print("Model test passed.")

## 5. Training: Conditional Video Diffusion with Classifier-Free Guidance

### TODO 5: Implement the Training Step

This is the core training logic. Classifier-free guidance dropout randomly replaces condition labels with -1 (unconditional), enabling controlled generation at inference. At Meridian, this means we can generate "dust storm" clips specifically, rather than random driving clips.

In [ ]:
def training_step(model, videos, labels, noise_schedule, p_uncond=0.1):
    """
    One training step of conditional video diffusion.

    Args:
        model: ConditionalVideoUNet
        videos: (B, C, T, H, W) clean dashcam clips
        labels: (B,) driving condition labels (0 to N_CLASSES-1)
        noise_schedule: Dict with 'alphas_cumprod' on correct device
        p_uncond: Probability of dropping condition (CFG)

    Returns:
        Scalar loss tensor

    Steps:
        1. Sample random timesteps t ~ Uniform(0, num_timesteps-1), shape (B,)
        2. Sample noise eps ~ N(0, I), same shape as videos
        3. Compute noisy videos:
           v_t = sqrt(alpha_bar_t) * v_0 + sqrt(1 - alpha_bar_t) * eps
           Important: reshape alpha_bar_t to (B, 1, 1, 1, 1) for broadcasting
        4. CFG dropout: clone labels, set labels[mask] = -1
           where mask = (torch.rand(B) < p_uncond)
        5. Predict noise: eps_pred = model(v_t, t, dropped_labels)
        6. Return F.mse_loss(eps_pred, eps)

    Hints:
        - ac = noise_schedule['alphas_cumprod']
        - ab = ac[t].reshape(-1, 1, 1, 1, 1) for broadcasting
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return torch.tensor(0.0)

In [ ]:
# Verification
model.train()
v_test = torch.randn(4, 3, 8, 64, 64).to(device)
l_test = torch.tensor([0, 1, 3, 5]).to(device)
loss = training_step(model, v_test, l_test, {
    'alphas_cumprod': noise_schedule['alphas_cumprod'].to(device)
})
assert loss.dim() == 0, "Loss should be scalar"
assert loss.item() > 0, "Loss should be positive"
assert loss.requires_grad, "Loss should have grad"
print(f"Test loss: {loss.item():.4f}")
print("TODO 5 passed.")

### Train on Real KITTI Data

In [ ]:
class DashcamDataset(Dataset):
    def __init__(self, videos, labels):
        # videos: (N, T, C, H, W), labels: (N,)
        self.videos = videos.permute(0, 2, 1, 3, 4)  # -> (N, C, T, H, W)
        self.labels = labels

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        return self.videos[idx], self.labels[idx]


model = ConditionalVideoUNet(n_classes=N_CLASSES).to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, eta_min=1e-5)
dataset = DashcamDataset(videos_tensor, labels_tensor)
loader = DataLoader(dataset, batch_size=8, shuffle=True, drop_last=True)
ns_device = {'alphas_cumprod': noise_schedule['alphas_cumprod'].to(device)}

losses = []
print(f"Training on {len(dataset)} real KITTI dashcam clips (40 epochs)...")
for epoch in range(40):
    epoch_loss = 0
    n = 0
    for vids, labs in loader:
        vids, labs = vids.to(device), labs.to(device)
        loss = training_step(model, vids, labs, ns_device, p_uncond=0.1)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        epoch_loss += loss.item()
        n += 1
    scheduler.step()
    avg = epoch_loss / max(n, 1)
    losses.append(avg)
    if (epoch + 1) % 10 == 0:
        print(f"  Epoch {epoch+1:3d}: Loss = {avg:.4f}, LR = {scheduler.get_last_lr()[0]:.2e}")

print("Training complete.")

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(losses, color='steelblue', linewidth=2)
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.title("Video Diffusion Training Loss on KITTI Dashcam Data")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Generating Synthetic Driving Clips

### Conditional Sampling with Classifier-Free Guidance

In [ ]:
@torch.no_grad()
def sample_videos(model, noise_schedule, labels, guidance_scale=3.0,
                  n_frames=8, size=64):
    """Sample driving clips conditioned on a driving condition label."""
    model.eval()
    B = labels.shape[0]
    betas = noise_schedule['betas'].to(device)
    alphas = noise_schedule['alphas'].to(device)
    ac = noise_schedule['alphas_cumprod'].to(device)
    T_steps = len(betas)

    x = torch.randn(B, 3, n_frames, size, size, device=device)

    for t in reversed(range(T_steps)):
        t_tensor = torch.full((B,), t, device=device)
        # CFG: conditional + unconditional
        eps_cond = model(x, t_tensor, labels)
        eps_uncond = model(x, t_tensor, torch.full_like(labels, -1))
        eps = eps_uncond + guidance_scale * (eps_cond - eps_uncond)

        alpha, ab, beta = alphas[t], ac[t], betas[t]
        mean = (1 / torch.sqrt(alpha)) * (x - beta / torch.sqrt(1 - ab) * eps)
        x = mean + (torch.sqrt(beta) * torch.randn_like(x) if t > 0 else 0)

    return x.clamp(0, 1)

### Generate Clips for Each Driving Condition

In [ ]:
conditions = ["City: moderate", "City: heavy", "City: intersections",
              "Residential: parked", "Residential: narrow", "Open road"]

print("Generating conditional dashcam clips...")
all_generated = {}
for cls in range(N_CLASSES):
    labels = torch.full((3,), cls, device=device)
    vids = sample_videos(model, noise_schedule, labels, guidance_scale=3.0)
    all_generated[cls] = vids.cpu()
    print(f"  Condition '{conditions[cls]}': 3 clips generated")

# Display
fig, axes = plt.subplots(N_CLASSES, 8, figsize=(20, 15))
fig.suptitle("Generated Driving Clips by Condition", fontsize=14, fontweight='bold')

for cls in range(N_CLASSES):
    vid = all_generated[cls][0]  # first clip
    for t in range(8):
        frame = vid[:, t].permute(1, 2, 0).numpy()
        axes[cls, t].imshow(frame)
        axes[cls, t].axis('off')
        if t == 0:
            axes[cls, t].set_ylabel(conditions[cls], fontsize=9, rotation=0,
                                     labelpad=100, va='center')

plt.tight_layout()
plt.show()

### Side-by-Side: Real vs. Generated

In [ ]:
fig, axes = plt.subplots(4, 8, figsize=(20, 10))
fig.suptitle("Real (rows 1,3) vs Generated (rows 2,4) Dashcam Clips", fontsize=14)

for pair_idx, cls in enumerate([0, 5]):  # city and open road
    # Real
    real_idx = (labels_tensor == cls).nonzero(as_tuple=True)[0][0]
    real_clip = videos_tensor[real_idx]
    # Generated
    gen_clip = all_generated[cls][0]

    for t in range(8):
        r = pair_idx * 2
        axes[r, t].imshow(real_clip[t].permute(1, 2, 0).numpy())
        axes[r, t].axis('off')
        if t == 0: axes[r, t].set_ylabel("Real", fontsize=11)

        axes[r+1, t].imshow(gen_clip[:, t].permute(1, 2, 0).numpy())
        axes[r+1, t].axis('off')
        if t == 0: axes[r+1, t].set_ylabel("Generated", fontsize=11)

plt.tight_layout()
plt.show()

## 7. Evaluation

### TODO 6: Frechet Video Distance

FVD is Meridian's primary quality metric. It measures distributional distance between real and generated video clips in a learned feature space. Target: FVD < 250.

In [ ]:
class SimpleVideoFeatureExtractor(nn.Module):
    """3D CNN feature extractor for FVD (fixed random weights)."""

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv3d(3, 32, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool3d((4, 4, 4)),
            nn.Flatten(),
            nn.Linear(32 * 64, 128),
        )
        for p in self.parameters():
            p.requires_grad = False

    def forward(self, x):
        return self.net(x)


def compute_fvd(real_videos, gen_videos, feature_extractor):
    """
    Compute Frechet Video Distance.

    Args:
        real_videos: (N, C, T, H, W) real clips
        gen_videos: (N, C, T, H, W) generated clips
        feature_extractor: Maps (B, C, T, H, W) -> (B, feat_dim)

    Returns:
        FVD score (float, lower = better)

    Steps:
        1. Extract features for real videos in batches of 16.
           Concatenate to get (N_real, feat_dim).
        2. Same for generated videos.
        3. Compute mu_real, sigma_real (mean and covariance)
        4. Compute mu_gen, sigma_gen
        5. FVD = ||mu_r - mu_g||^2 + Tr(S_r + S_g - 2 * sqrtm(S_r @ S_g))
           Use scipy.linalg.sqrtm for the matrix square root.
           Add 1e-6 * I for numerical stability.

    Hints:
        - features.mean(dim=0) for mu
        - Center features, then X^T @ X / (N-1) for covariance
        - scipy.linalg.sqrtm may return complex; take .real
    """
    from scipy import linalg
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return 0.0

In [ ]:
# Compute FVD
feat_ext = SimpleVideoFeatureExtractor().to(device)

# Collect generated clips
gen_all = torch.cat([all_generated[c] for c in range(N_CLASSES)], dim=0)
gen_all = gen_all.to(device)

# Sample real clips (same count)
n_gen = gen_all.shape[0]
real_sample = videos_tensor[:n_gen].permute(0, 2, 1, 3, 4).to(device)

fvd = compute_fvd(real_sample, gen_all, feat_ext)
print(f"FVD score: {fvd:.1f}")
print(f"Meridian target: < 250")
print(f"{'PASS' if fvd < 250 else 'Above threshold -- model needs more training or data'}")
print("TODO 6 passed." if fvd > 0 else "TODO 6 needs work.")

## 8. Error Analysis: Driving-Specific Failure Modes

### TODO 7: Categorize Failures in Generated Driving Clips

At Meridian, every generated clip passes through an automated quality gate before being added to the training pool. We check for driving-specific artifacts that would corrupt perception model training.

In [ ]:
def driving_error_analysis(generated_clips, condition_labels, n_inspect=18):
    """
    Categorize failure modes in generated driving clips.

    Args:
        generated_clips: dict mapping condition (int) -> (N, C, T, H, W) clips
        condition_labels: list of condition name strings
        n_inspect: total clips to inspect

    Returns:
        Dict mapping failure_type (str) to count (int):
            'temporal_flicker': max frame-to-frame pixel diff > 0.4
            'static_video': mean frame-to-frame diff < 0.01 (no motion at all)
            'color_drift': mean color of first frame differs from last by > 0.15
            'clean': no failures detected

    Steps:
        1. Collect all generated clips into a single list
        2. Randomly select n_inspect clips
        3. For each clip (C, T, H, W):
           a. Frame diffs: diff = (clip[:, 1:] - clip[:, :-1]).abs()
              max_diff = diff.max().item()
              mean_diff = diff.mean().item()
           b. Color drift: |clip[:, 0].mean() - clip[:, -1].mean()|
        4. Categorize based on thresholds
        5. Count and return
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return {}

In [ ]:
# Run error analysis
failures = driving_error_analysis(all_generated, conditions)
print("Driving-Specific Failure Analysis:")
print("-" * 40)
for cat, count in sorted(failures.items(), key=lambda x: -x[1]):
    print(f"  {cat:<20} {count:>3} clips")

total = sum(failures.values())
clean = failures.get('clean', 0)
reject_rate = (total - clean) / max(total, 1) * 100
print(f"\nRejection rate: {reject_rate:.0f}%")
print(f"Meridian target: < 15% rejection")
print("TODO 7 passed.")

**Think about this:** At Meridian, each failure mode has different downstream impact:
- *Temporal flicker* causes the object tracker to fragment, losing track continuity
- *Static video* provides no motion learning signal (the perception model sees a photograph)
- *Color drift* teaches the model to expect impossible lighting changes, corrupting night/weather detection

Which failure mode would you prioritize fixing first? What architectural change would you try?

## 9. Inference Benchmarking

### TODO 8: Profile Generation Latency

Meridian needs to generate 10,000 clips per week. Profile the inference pipeline to determine GPU requirements.

In [ ]:
def benchmark_generation(model, noise_schedule, n_runs=3):
    """
    Benchmark video generation latency.

    Args:
        model: Trained video diffusion model
        noise_schedule: Schedule dict
        n_runs: Number of runs to average

    Returns:
        Dict with 'mean_seconds', 'std_seconds', 'per_step_ms', 'peak_memory_mb'

    Steps:
        1. Warm up with 1 dummy generation
        2. For each run:
           a. torch.cuda.synchronize() before starting timer
           b. torch.cuda.reset_peak_memory_stats()
           c. Generate one clip using sample_videos (single clip, guidance=3.0)
           d. torch.cuda.synchronize() after generation
           e. Record time and peak memory
        3. Compute mean, std, per-step time, peak memory
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return {'mean_seconds': 0.0, 'std_seconds': 0.0,
            'per_step_ms': 0.0, 'peak_memory_mb': 0.0}

In [ ]:
bench = benchmark_generation(model, noise_schedule)
print(f"Generation time: {bench['mean_seconds']:.1f}s (+/- {bench['std_seconds']:.1f}s)")
print(f"Per-step latency: {bench['per_step_ms']:.1f}ms")
print(f"Peak GPU memory: {bench['peak_memory_mb']:.0f} MB")

# Capacity planning for Meridian
clips_per_hour = 3600 / max(bench['mean_seconds'], 0.1)
clips_per_week_1gpu = clips_per_hour * 24 * 7
gpus_needed = 10000 / max(clips_per_week_1gpu, 1)
print(f"\nCapacity planning:")
print(f"  {clips_per_hour:.0f} clips/hour per GPU")
print(f"  {clips_per_week_1gpu:.0f} clips/week per GPU (24/7)")
print(f"  {gpus_needed:.1f} GPUs needed for 10,000 clips/week")
print(f"\nOptimizations to consider: DDIM (50 steps instead of 200),")
print(f"  half-precision (fp16), model distillation, batch generation")
print("TODO 8 passed.")

## 10. Ethical and Bias Analysis

### TODO 9: Per-Condition Quality Audit

At Meridian, if the model generates "nighttime" scenarios with lower quality than "daytime," the perception model trained on that data will have a systematic blind spot at night -- exactly when safety matters most. Check for this bias.

In [ ]:
def per_condition_quality_audit(model, noise_schedule, n_per_class=5):
    """
    Audit generation quality bias across driving conditions.

    Args:
        model: Trained model
        noise_schedule: Schedule dict
        n_per_class: Clips per condition

    Returns:
        Dict with:
            'per_condition_coherence': {cls: float}
            'per_condition_sharpness': {cls: float} (mean Sobel gradient magnitude)
            'worst_condition': int
            'best_condition': int
            'quality_gap': float (best - worst coherence)

    Steps:
        1. For each condition 0..N_CLASSES-1:
           a. Generate n_per_class clips
           b. Compute temporal coherence (reuse temporal_coherence_score)
           c. Compute sharpness: for each frame, apply Sobel filter
              Sobel x kernel: [[-1,0,1],[-2,0,2],[-1,0,1]]
              Sharpness = mean |Sobel_x| + |Sobel_y| across all frames
           d. Average coherence and sharpness for this condition
        2. Identify best/worst conditions
    """
    # ============ TODO ============
    #
    # YOUR CODE HERE
    #
    # ==============================
    return {
        'per_condition_coherence': {},
        'per_condition_sharpness': {},
        'worst_condition': 0,
        'best_condition': 0,
        'quality_gap': 0.0,
    }

In [ ]:
audit = per_condition_quality_audit(model, noise_schedule, n_per_class=3)
conditions = ["City: moderate", "City: heavy", "City: intersections",
              "Residential: parked", "Residential: narrow", "Open road"]

print("Per-Condition Generation Quality:")
print(f"{'Condition':<25} {'Coherence':>10} {'Sharpness':>10}")
print("-" * 50)
for cls in range(N_CLASSES):
    coh = audit['per_condition_coherence'].get(cls, 0)
    sharp = audit['per_condition_sharpness'].get(cls, 0)
    marker = " <-- worst" if cls == audit['worst_condition'] else ""
    print(f"{conditions[cls]:<25} {coh:>10.3f} {sharp:>10.4f}{marker}")

print(f"\nQuality gap: {audit['quality_gap']:.3f}")
print(f"Worst condition: {conditions[audit['worst_condition']]}")
print("TODO 9 passed.")

**Critical questions for Meridian's ML team:**
- If the model generates "open road" clips with significantly lower quality than "city" clips, what happens when the perception model is deployed on highway routes?
- How would you enforce that ALL driving conditions meet a minimum quality threshold before synthetic data enters the training pipeline?
- What SOC 2 controls would you place around the synthetic data pipeline? Consider: data provenance tracking, model versioning, and audit logs.

## Summary and Next Steps

You have built an end-to-end synthetic driving video generation pipeline:

1. **Downloaded real dashcam data** from KITTI -- industry-standard autonomous driving benchmark
2. **Analyzed temporal dynamics** across driving conditions (city, residential, highway)
3. **Proved the baseline fails** -- frame-independent generation has no temporal coherence
4. **Built factorized attention** -- spatial attention per frame + temporal attention across frames
5. **Trained with classifier-free guidance** -- enabling condition-controlled generation
6. **Evaluated with FVD** -- the standard metric for video generation quality
7. **Analyzed driving-specific failure modes** -- flicker, static video, color drift
8. **Benchmarked inference** for Meridian's 10,000 clips/week production target
9. **Audited per-condition quality bias** to prevent systematic blind spots

### Scaling This to Production

The model you trained operates at 64x64 resolution on 8 frames. Meridian's production system generates at 1280x720 on 40 frames. To bridge this gap:

- **Latent diffusion**: Compress dashcam frames to a 16x smaller latent space via a pretrained VAE, then run diffusion in latent space (see Section 2 of the case study)
- **Cascaded upsampling**: Generate at low resolution first, then upscale with a super-resolution model (Imagen Video approach)
- **Pretrained initialization**: Start from Stable Diffusion's spatial layers, train only the temporal layers on driving data (Stable Video Diffusion approach)
- **DDIM sampling**: Reduce from 200 to 50 reverse steps for 4x faster inference

### Using Larger Driving Datasets

KITTI is excellent for prototyping but limited in weather diversity. For production-scale training:

- **nuScenes** (nuscenes.org): 1000 scenes from Boston and Singapore with rain, night, and diverse conditions. Register (free) and download the mini split (4 GB) or full trainval (60 GB).
- **BDD100K** (bdd-data.berkeley.edu): 100K driving clips with explicit weather and time-of-day labels. Ideal for training weather-conditioned generation.
- **Waymo Open** (waymo.com/open): High-quality driving data from Waymo's fleet across multiple US cities.

For **Section 4 (Production Design)** of the full case study -- API design, serving infrastructure, monitoring, A/B testing, and cost analysis -- refer to the case study PDF.